# PymuPDF reader
- Read the tutorial here: https://pymupdf.readthedocs.io/en/latest/tutorial.html

In [1]:
# automatically reload modules, including the ones you wrote yourself
%load_ext autoreload
%autoreload 2

import fitz
import re

import pdf_processor as ppr

print(f"Using PyMuPDF v{fitz.__version__} from {fitz.__file__}")


Using PyMuPDF v1.26.3 from /Users/chandlershortlidge/Desktop/Ironhack/resume-reader/.venv/lib/python3.12/site-packages/fitz/__init__.py


In [12]:
RESUME_PATH = "Chandler Shortlidge UXW CV 2025.pdf"

In [13]:
doc = ppr.load_file(RESUME_PATH)
ppr.report_metadata(doc)

Loaded document with 2 pages
format: PDF 1.4
title: Chandler Shortlidge UXW CV 2025
author: 
subject: 
keywords: 
creator: 
producer: Skia/PDF m140 Google Docs Renderer
creationDate: 
modDate: 
trapped: 
encryption: None


### Keyword Searching
- To locate where a term appears on the page (useful for highlighting)

In [14]:
# Define the keywords you want to find
keywords_to_find = ["Python", "SQL", "AWS", "Data Scientist"]

# Call your new function to highlight the keywords
doc = ppr.highlight_keywords(doc, keywords_to_find)

# save PDF with highlights
ppr.save_highlighted_doc(doc, "my_highlighted_resume.pdf")              
  

Saved highlighted PDF as: my_highlighted_resume.pdf


In [15]:
# Call the function, which returns a single formatted string
markdown_output = ppr.pdf_to_markdown(doc)

# Write the string to an .md file and receive output "Markdown content successfully saved to: file_name.md"
output_filename = "CS_UX.md"
ppr.save_markdown_file(markdown_output, output_filename)

Markdown content successfully saved to CS_UX.md


### Get links

In [16]:
ppr.get_links(doc)
 


Page 1 links:
  -> URI: https://ninth-moose-2d2.notion.site/Chandler-Shortlidge-UXW-Portfolio-e9aa7c37c14244f286808939928a3c66
    location on page: Rect(223.84927368164062, 83.54998779296875, 303.27789306640625, 95.54998779296875)
  -> URI: mailto:chandlershortlidge@gmail.com
    location on page: Rect(382.9499816894531, 88.04998779296875, 526.1500244140625, 98.8499755859375)
  -> URI: https://buycycle.com/en-de
    location on page: Rect(113.62742614746094, 172.29998779296875, 162.7513885498047, 185.5)
  -> URI: https://www.beatportal.com/
    location on page: Rect(141.84478759765625, 364.29998779296875, 191.5516357421875, 377.5)
  -> URI: https://djmag.com/
    location on page: Rect(125.13646697998047, 556.300048828125, 167.30255126953125, 569.5000610351562)
  -> URI: mailto:jonas@buycycle.de
    location on page: Rect(382.9499816894531, 478.0500183105469, 436.2753601074219, 488.8500061035156)
  -> URI: mailto:stephenflynn17@gmail.com
    location on page: Rect(382.9499816894531, 

### Extracting Text and Images

In [ ]:
# Extract text from each page

for page_num in range(doc.page_count):
    page = doc.load_page(page_num)
    text = page.get_text()

#close when done
# doc.close()

text